## Exercises

### Exercise 1

A) Write a program implementing a linear congruential generator (LCG). Be sure that the program works correctly using only integer representation.

**Linear Congruential Generator (LCG)**

An LCG has the form:

$$
X_{n+1} = (a \cdot X_n + c) \mod m
$$

Where:

- \( a \): multiplier  
- \( c \): shift 
- \( m \): modulus  
- \( X_0 \): initial value (seed)


In [2]:
# LCG parameters
# Linear Congruential Generator (LCG) implementation
a = 5
c = 1
m = 16
seed = 237

def lcg(n, seed=seed):
    x = seed
    numbers = []
    for _ in range(n):
        x = (a * x + c) % m
        numbers.append(x / m)  # Normaliser til [0,1)
    return numbers

# Generate 10000 random numbers
numbers = lcg(10000)

# Parameters for [0,1) interval
num_bins = 10
min_val = 0.0
max_val = 1.0
bin_width = (max_val - min_val) / num_bins

# Initialize bins
hist_counts = [0] * num_bins

# Count occurrences in each bin
for val in numbers:
    bin_index = int((val - min_val) / bin_width)
    if bin_index >= num_bins:  # Clamp to last bin if val == 1.0 (rare but safe)
        bin_index = num_bins - 1
    hist_counts[bin_index] += 1

# Determine max count for scaling
max_count = max(hist_counts)

# Build text histogram
histogram_text = ""
for i in range(num_bins):
    start = min_val + i * bin_width
    end = start + bin_width
    bar = '█' * int((hist_counts[i] / max_count) * 40)
    label = f"[{start:.1f}, {end:.1f})"
    histogram_text += f"{label.ljust(14)} | {bar} ({hist_counts[i]})\n"

print(histogram_text)



[0.0, 0.1)     | ████████████████████████████████████████ (1250)
[0.1, 0.2)     | ████████████████████████████████████████ (1250)
[0.2, 0.3)     | ████████████████████ (625)
[0.3, 0.4)     | ████████████████████████████████████████ (1250)
[0.4, 0.5)     | ████████████████████ (625)
[0.5, 0.6)     | ████████████████████████████████████████ (1250)
[0.6, 0.7)     | ████████████████████████████████████████ (1250)
[0.7, 0.8)     | ████████████████████ (625)
[0.8, 0.9)     | ████████████████████████████████████████ (1250)
[0.9, 1.0)     | ████████████████████ (625)



#### B

Evaluate the quality of the generator by graphical descriptive statistics (histogrammes, scatter plots) and statistical tests- χ2,Kolmogorov-Smirnov, run-tests preferably but not necessarily all 3, and correlation test forsome h-values.

In [9]:
# Show 20 x_i
print(" i     X_i     ")
print("--------------")
for i in range(20):
    xi = round(numbers[i], 4)
    print(f"{i:<5} {xi:<10} ")


 i     X_i     
--------------
0     0.125      
1     0.6875     
2     0.5        
3     0.5625     
4     0.875      
5     0.4375     
6     0.25       
7     0.3125     
8     0.625      
9     0.1875     
10    0.0        
11    0.0625     
12    0.375      
13    0.9375     
14    0.75       
15    0.8125     
16    0.125      
17    0.6875     
18    0.5        
19    0.5625     


In [4]:
expected = len(numbers) // num_bins
chi_squared = 0
for observed in hist_counts:
    chi_squared += ((observed - expected) ** 2) / expected

print(f"\nChi-squared statistic: {chi_squared:.2f}")



Chi-squared statistic: 937.50


In [5]:
run_count = 1
prev_trend = None
short_seq = numbers[:100]  # take first 100 values
for i in range(1, len(short_seq)):
    if short_seq[i] > short_seq[i - 1]:
        trend = "up"
    elif short_seq[i] < short_seq[i - 1]:
        trend = "down"
    else:
        trend = "same"

    if trend != prev_trend and trend != "same":
        run_count += 1
        prev_trend = trend

print(f"\nEstimated number of runs (first 100 values): {run_count}")



Estimated number of runs (first 100 values): 64


In [6]:
n = len(numbers) - 1
x = numbers[:-1]
y = numbers[1:]
mean_x = sum(x) / n
mean_y = sum(y) / n

numerator = sum((x[i] - mean_x) * (y[i] - mean_y) for i in range(n))
denominator_x = sum((x[i] - mean_x) ** 2 for i in range(n))
denominator_y = sum((y[i] - mean_y) ** 2 for i in range(n))
correlation = numerator / (denominator_x * denominator_y) ** 0.5

print(f"\nCorrelation between X_i and X_(i+1): {correlation:.4f}")



Correlation between X_i and X_(i+1): 0.2708


In [ ]:
Lottermann = 0